<a href="https://colab.research.google.com/github/AlirezaAhadipour/Practice/blob/main/gnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Graph Neural Netrorks

perform node classification on **Cora** benchmark.

- citation networks classify papers into topics
- 2708 nodes, 5429 edges
- bag-of-words features, 7 topics
- 140 training, 500 validation, 1000 test

In [1]:
!pip install -q spektral --upgrade

In [2]:
import numpy as np
import tensorflow as tf
import spektral

# from spektral.datasets import citation

In [3]:
cora_dataset = spektral.datasets.citation.Citation(name='cora')

graph = cora_dataset.graphs[0]
adjacency = graph.a
features = graph.x
labels = graph.y

train_mask = cora_dataset.mask_tr
val_mask = cora_dataset.mask_va
test_mask = cora_dataset.mask_te

In [4]:
adjacency = adjacency + np.eye(adjacency.shape[0])
adjacency = adjacency.astype('float32')

features = features.astype('float32')

In [5]:
print(adjacency.shape)
print(features.shape)
print(labels.shape)

(2708, 2708)
(2708, 1433)
(2708, 7)


In [6]:
print(train_mask.sum())

140


In [7]:
def masked_softmax_cross_entropy(logits, labels, mask):
  loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)

  mask = tf.cast(mask, dtype=tf.float32)
  mask /= tf.reduce_mean(mask)

  loss *= mask

  return tf.reduce_mean(loss)


def masked_accuracy(logits, labels, mask):
  correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
  accuracy_all = tf.cast(correct_prediction, dtype=tf.float32)

  mask = tf.cast(mask, dtype=tf.float32)
  mask /= tf.reduce_mean(mask)

  accuracy_all *= mask

  return tf.reduce_mean(accuracy_all)

In [8]:
def gnn(adjacency_mtrx, feature_mtrx, transformation, activation_fn):
  seq_fts = transformation(feature_mtrx)
  ret_fts = tf.matmul(adjacency_mtrx, seq_fts)

  return(activation_fn(ret_fts))

In [9]:
def train_cora(adj, fts, gnn_fn, units, epochs, lr):
  lyr_1 = tf.keras.layers.Dense(units)
  lyr_2 = tf.keras.layers.Dense(7)

  def gnn_cora(adj, fts):
    hidden = gnn_fn(adj, fts, lyr_1, tf.nn.relu)
    logits = gnn_fn(adj, hidden, lyr_2, tf.identity)

    return logits

  optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

  best_accuracy = 0
  for epoch in range(epochs + 1):
    with tf.GradientTape() as t:
      logits = gnn_cora(adj, fts)
      loss = masked_softmax_cross_entropy(logits, labels, train_mask)

    vars = t.watched_variables()
    grads = t.gradient(loss, vars)
    optimizer.apply_gradients(zip(grads, vars))

    logits = gnn_cora(adj, fts)
    val_accuracy = masked_accuracy(logits, labels, val_mask)
    test_accuracy = masked_accuracy(logits, labels, test_mask)

    if val_accuracy > best_accuracy:
      best_accuracy = val_accuracy
      print(f"Epoch: {epoch}, train_loss: {loss.numpy():.4f}, val_accuracy: {val_accuracy.numpy():.4f}, test_accuracy:  {test_accuracy.numpy():.4f}")

In [10]:
# sum pooling
train_cora(adjacency, features, gnn, 32, 100, 0.01)

Epoch: 0, train_loss: 7.2104, val_accuracy: 0.1940, test_accuracy:  0.1970
Epoch: 1, train_loss: 5.4446, val_accuracy: 0.2620, test_accuracy:  0.2650
Epoch: 2, train_loss: 3.7592, val_accuracy: 0.5780, test_accuracy:  0.6050
Epoch: 3, train_loss: 2.0005, val_accuracy: 0.6280, test_accuracy:  0.6720
Epoch: 6, train_loss: 1.0376, val_accuracy: 0.6440, test_accuracy:  0.6530
Epoch: 11, train_loss: 0.2765, val_accuracy: 0.6540, test_accuracy:  0.6710
Epoch: 13, train_loss: 0.1826, val_accuracy: 0.6760, test_accuracy:  0.7090
Epoch: 14, train_loss: 0.1585, val_accuracy: 0.6840, test_accuracy:  0.7270
Epoch: 15, train_loss: 0.1400, val_accuracy: 0.6880, test_accuracy:  0.7330
Epoch: 16, train_loss: 0.1254, val_accuracy: 0.6980, test_accuracy:  0.7340
Epoch: 17, train_loss: 0.1141, val_accuracy: 0.7020, test_accuracy:  0.7410
Epoch: 18, train_loss: 0.1039, val_accuracy: 0.7100, test_accuracy:  0.7380
Epoch: 19, train_loss: 0.0928, val_accuracy: 0.7140, test_accuracy:  0.7370
Epoch: 38, train_

In [11]:
# mean pooling
degree = tf.reduce_sum(adjacency, axis=-1)

train_cora(adjacency/degree, features, gnn, 32, 100, 0.01)

Epoch: 0, train_loss: 1.9327, val_accuracy: 0.2660, test_accuracy:  0.3190
Epoch: 1, train_loss: 1.7101, val_accuracy: 0.4220, test_accuracy:  0.4410
Epoch: 2, train_loss: 1.4737, val_accuracy: 0.5260, test_accuracy:  0.5460
Epoch: 3, train_loss: 1.2409, val_accuracy: 0.6240, test_accuracy:  0.6480
Epoch: 4, train_loss: 1.0277, val_accuracy: 0.6940, test_accuracy:  0.7180
Epoch: 5, train_loss: 0.8406, val_accuracy: 0.7360, test_accuracy:  0.7560
Epoch: 6, train_loss: 0.6791, val_accuracy: 0.7640, test_accuracy:  0.7790
Epoch: 7, train_loss: 0.5436, val_accuracy: 0.7820, test_accuracy:  0.8000
Epoch: 9, train_loss: 0.3441, val_accuracy: 0.7860, test_accuracy:  0.8080
Epoch: 11, train_loss: 0.2146, val_accuracy: 0.7880, test_accuracy:  0.8070
Epoch: 12, train_loss: 0.1691, val_accuracy: 0.7940, test_accuracy:  0.8060


In [12]:
# graph convolutional network
deg = tf.reduce_sum(adjacency, axis=-1)
norm_deg = tf.linalg.diag(1.0 / tf.sqrt(deg))
norm_adj = tf.matmul(norm_deg, tf.matmul(adjacency, norm_deg))

train_cora(norm_adj, features, gnn, 32, 100, 0.01)

Epoch: 0, train_loss: 1.9479, val_accuracy: 0.6420, test_accuracy:  0.6420
Epoch: 1, train_loss: 1.7555, val_accuracy: 0.7140, test_accuracy:  0.7310
Epoch: 2, train_loss: 1.5274, val_accuracy: 0.7460, test_accuracy:  0.7830
Epoch: 3, train_loss: 1.2857, val_accuracy: 0.7580, test_accuracy:  0.7940
Epoch: 4, train_loss: 1.0600, val_accuracy: 0.7620, test_accuracy:  0.7990
Epoch: 5, train_loss: 0.8587, val_accuracy: 0.7820, test_accuracy:  0.8050
Epoch: 6, train_loss: 0.6821, val_accuracy: 0.7840, test_accuracy:  0.8070
